In [1]:
from gensim import corpora, models, similarities
from gensim.models import hdpmodel, ldamodel
from itertools import izip
from nltk.corpus import stopwords
from pandas.core.series import Series
import math
from nltk.stem import WordNetLemmatizer
import pdb
import pandas as pd

/Users/himavarsha/Envs/sklearn/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
documents = []
filename = "NewScientist-Data_sent_output.xlsx"
df = pd.read_excel(filename, "Data")
documents = df['Text'].tolist()

In [3]:
stoplist = stopwords.words("english") + ['blackberry','it\'s','i','a','an','is','my','to','it','the','oven']


In [4]:
wordnet_lemmatizer = WordNetLemmatizer()
texts = []
for document in documents:
    words = document.lower().replace("."," ").split()
    text_sample = []
    for word in words:
        try:
            if wordnet_lemmatizer.lemmatize(word) not in stoplist:
                text_sample.append(wordnet_lemmatizer.lemmatize(word))
        except:
            pdb.set_trace()
    texts.append(text_sample)
# texts = [[wordnet_lemmatizer.lemmatize(word) for word in document.lower().split() if word not in stoplist] for document in documents]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [5]:
for topic_num in xrange(num_topics):
    print "Topic " + str(topic_num) + str(lda.print_topics(num_topics)[topic_num])

NameError: name 'num_topics' is not defined

In [ ]:
lda.print_topics(num_topics)[topic_num]

In [ ]:
probs = lda[corpus]

In [ ]:
len(probs)

In [6]:
import numpy as np
probabilities = np.zeros((len(documents),num_topics))
for prob_ind in xrange(len(probs)):
    for prob_item in probs[prob_ind]:
        probabilities[prob_ind][prob_item[0]] = prob_item[1]


NameError: name 'num_topics' is not defined

In [7]:
# pd.DataFrame(probabilities)
final_df = pd.concat([pd.DataFrame(documents),pd.DataFrame(probabilities)],axis=1)

NameError: name 'probabilities' is not defined

In [11]:
from gensim.matutils import cossim
def intra_inter(model, test_docs, num_pairs=10000):
    # split each test document into two halves and compute topics for each half
    part1 = [model[dictionary.doc2bow(tokens[: len(tokens) / 2])] for tokens in test_docs]
    part2 = [model[dictionary.doc2bow(tokens[len(tokens) / 2 :])] for tokens in test_docs]
    
    # print computed similarities (uses cossim)
#     print("average cosine similarity between corresponding parts (higher is better):")
    sim = (np.mean([cossim(p1, p2) for p1, p2 in zip(part1, part2)]))
#     print sim

    random_pairs = np.random.randint(0, len(test_docs), size=(num_pairs, 2))
#     print("average cosine similarity between 10,000 random parts (lower is better):")    
    non_sim = (np.mean([cossim(part1[i[0]], part2[i[1]]) for i in random_pairs]))
#     print non_sim
    return sim, non_sim 

In [9]:
num_topics = 10
lda = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, update_every=5, chunksize=10000, passes=10)

KeyboardInterrupt: 

In [ ]:
print("LDA results:")
intra_inter(lda, texts)

In [17]:
max_sim = 0
max_non_sim = 1
optimal_topics = 0
for num_topics in xrange(2,100):
    print(num_topics)
    lda_model = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=num_topics)
    sim, non_sim = intra_inter(lda_model,texts)
    
    if (sim >= max_sim + 0.25 and non_sim <= max_non_sim + 0.25) or (non_sim <= max_non_sim - 0.25 and sim >= max_sim - 0.25):
        optimal_topics = num_topics
        if sim >= max_sim:
            max_sim = sim
        if non_sim <= max_non_sim:
            max_non_sim = non_sim
print optimal_topics,max_sim,max_non_sim

2
average cosine similarity between corresponding parts (higher is better):
0.920458835719
average cosine similarity between 10,000 random parts (lower is better):
0.792309178432
3
average cosine similarity between corresponding parts (higher is better):
0.882067082721
average cosine similarity between 10,000 random parts (lower is better):
0.596030705071
4
average cosine similarity between corresponding parts (higher is better):
0.834717739631
average cosine similarity between 10,000 random parts (lower is better):
0.47873817283
5
average cosine similarity between corresponding parts (higher is better):
0.80336457534
average cosine similarity between 10,000 random parts (lower is better):
0.413037256316
6
average cosine similarity between corresponding parts (higher is better):
0.786360114095
average cosine similarity between 10,000 random parts (lower is better):
0.361562486977
7
average cosine similarity between corresponding parts (higher is better):
0.758403760655
average cosine s

In [ ]:
for i in xrange(num_topics):
    